In [1]:
import pandas as pd
import numpy as np
import artm
from tqdm import tqdm_notebook
import time
from proress_bar import log_progress
%load_ext autotime

In [18]:
with open('football.txt') as fin:
    logs = fin.readlines()

In [7]:
logs

NameError: name 'logs' is not defined

time: 7.98 ms


In [49]:
text = []
for line in tqdm_notebook(logs[1:]):
    line = line.split('\t')
    new_line = '|text ' + line[0] + ' \n'
    text.append(new_line)

print(text[:10])


['|text datakam \n', '|text yuxuda kız usagi gormek \n', '|text порно \n', '|text цефотаксим \n', '|text вертикальном цилиндрическом сосуде над поршнем а находится газ \n', '|text панель управления стиральной машины самсунг эко бабл \n', '|text полки авито екатеринбург \n', '|text зарплата санитарки в больнице боткина \n', '|text аптека в кубинке торговля травами \n', '|text wapsky ru \n']


In [53]:
with open('logs.txt', 'w') as fout:
    for line in tqdm_notebook(text): 
        fout.write(line)

In [3]:
%time
bach_vectorizer = artm.BatchVectorizer(data_path='logs.txt', data_format='vowpal_wabbit', target_folder='batch',
                                      batch_size=20)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [8]:
T = 30   # количество тем
topic_names=["sbj"+str(i) for i in range(T-1)]+["bcg"]

model_artm = artm.ARTM(num_topics=T, topic_names=topic_names, reuse_theta=True,
                       num_document_passes=1)

time: 5.02 ms


In [9]:
np.random.seed(1)
dictionary = artm.Dictionary()
log_progress(dictionary.gather(data_path=bach_vectorizer.data_path))
log_progress(model_artm.initialize(dictionary))

<generator object log_progress at 0x142426de0>

time: 27min 33s


In [10]:
model_artm.scores.add(artm.TopTokensScore(name='metric1', num_tokens=15))

time: 1.01 ms


In [11]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='smoothing', dictionary=dictionary,
                                                         topic_names='bcg', tau=1e5))

time: 1.32 ms


In [ ]:
model_artm.fit_offline(batch_vectorizer=bach_vectorizer, num_collection_passes=1)

In [ ]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='stimulates',
                                                            dictionary=dictionary,
                                                            topic_names=["sbj"+str(i) for i in range(0, 29)], tau=-1e5))

In [ ]:
model_artm.fit_offline(batch_vectorizer=bach_vectorizer, num_collection_passes=1)

In [ ]:
for topic_name in model_artm.topic_names:
    print topic_name + ': ',
    for word in model_artm.score_tracker["metric1"].last_tokens[topic_name]:
        print word,
    print